In [24]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [25]:
data = Dataset.load_builtin('ml-100k')
train_set, test_set = train_test_split(data, test_size=.25, random_state=0)


In [26]:
algo = SVD()
algo.fit(train_set)

In [27]:
predictions = algo.test(test_set)
print('type(predictions): ', type(predictions), 'len(predictions):', len(predictions))

predictions[:5]

type(predictions):  <class 'list'> len(predictions): 25000


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.677447270262077, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7509888423370104, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=3.943027746284129, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.394969043132595, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.53246507636709, details={'was_impossible': False})]

In [28]:
[ (pred.uid, pred.iid, pred.est )for pred in predictions[:3] ]

[('120', '282', 3.677447270262077),
 ('882', '291', 3.7509888423370104),
 ('535', '507', 3.943027746284129)]

In [29]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.01   {'was_impossible': False}


In [30]:
accuracy.rmse(predictions)

RMSE: 0.9472


0.9472387188415261

In [31]:
import pandas as pd

ratings = pd.read_csv('ratings.csv')

ratings.to_csv('ratings_without_header.csv', index=False, header=False)

In [32]:
from surprise import Reader

reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5))
data = Dataset.load_from_file('ratings_without_header.csv', reader=reader)

In [34]:
train_set, test_set = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(train_set)
predictions = algo.test(test_set)

accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [36]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
train_set, test_set = train_test_split(data, test_size=.25, random_state=0)

algorithm = SVD(n_factors=50, random_state=0)
algorithm.fit(train_set)
predictions = algorithm.test(test_set)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [37]:
from surprise.model_selection import cross_validate

ratings = pd.read_csv('ratings.csv')
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

algorithm = SVD(random_state=0)
cross_validate(algorithm, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8753  0.8736  0.8707  0.8736  0.8697  0.8726  0.0021  
MAE (testset)     0.6713  0.6690  0.6710  0.6739  0.6675  0.6705  0.0022  
Fit time          4.90    4.88    4.89    4.82    4.88    4.87    0.03    
Test time         0.33    0.12    0.12    0.12    0.31    0.20    0.10    


{'test_rmse': array([0.87527876, 0.87357   , 0.87071634, 0.87360062, 0.86970407]),
 'test_mae': array([0.6712662 , 0.66896049, 0.67099928, 0.67386942, 0.66753673]),
 'fit_time': (4.895795822143555,
  4.877290964126587,
  4.891078948974609,
  4.822567939758301,
  4.876069068908691),
 'test_time': (0.32698917388916016,
  0.12301278114318848,
  0.12084197998046875,
  0.12226724624633789,
  0.3062567710876465)}

In [38]:
from surprise.model_selection import GridSearchCV

param_grid = {
    'n_epochs': [20, 40, 60],
    'n_factors': [50, 100, 200]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8757935925364245
{'n_epochs': 20, 'n_factors': 50}


In [39]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
algorithm = SVD(n_factors=50, random_state=0)
algorithm.fit(data)

AttributeError: 'DatasetAutoFolds' object has no attribute 'global_mean'

In [42]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format='user item rating timestamp', sep=',',rating_scale=(0.5, 5))

data_folds = DatasetAutoFolds(ratings_file='ratings_without_header.csv', reader=reader)

train_set = data_folds.build_full_trainset()

algorithm = SVD(n_epochs=20, n_factors=50, random_state=0)
algorithm.fit(train_set)

In [44]:
movies = pd.read_csv('movies.csv')

movie_ids = ratings[ratings['userId'] == 9]['movieId']

if movie_ids[movie_ids == 42].count() == 0:
    print(f'there is no rating of movieId 42 of userId 9')

print(movies[movies['movieId'] == 42])

there is no rating of movieId 42 of userId 9
    movieId                   title              genres
38       42  Dead Presidents (1995)  Action|Crime|Drama


In [45]:
uid = '9'
iid = '42'

pred = algorithm.predict(uid, iid, verbose=True)

user: 9          item: 42         r_ui = None   est = 3.13   {'was_impossible': False}


In [47]:
def get_unseen_surprise(ratings, movies, userId):
    seen_movies = ratings[ratings['userId'] == userId]['movieId'].tolist()

    total_movies = movies['movieId'].tolist()

    unseen_movies = [movie for movie in total_movies if movie not in seen_movies]

    print('len(seen_movies): ', len(seen_movies))
    print('len(seen_movies): ', len(unseen_movies))
    print('len(seen_movies): ', len(total_movies))

    return unseen_movies

unseen_movies = get_unseen_surprise(ratings, movies, 9)

len(seen_movies):  46
len(seen_movies):  9696
len(seen_movies):  9742


In [49]:
def recomment_movie_by_surprise(algorithm, userId, unseen_movies, top_n=10):

    predictions = [algorithm.predict(str(userId), str(movieId)) for movieId in unseen_movies]

    def sortkey_est(pred):
        return pred.est

    predictions.sort(key=sortkey_est, reverse=True)
    top_predictions = predictions[:top_n]

    top_movies_ids = [int(pred.iid) for pred in top_predictions]
    top_movies_ratings = [pred.est for pred in top_predictions]
    top_movies_titles = movies[movies.movieId.isin(top_movies_ids)]['title']

    top_movie_preds = [ (id, title, rating) for id, title, rating in zip(top_movies_ids, top_movies_titles, top_movies_ratings) ]

    return top_movie_preds

unseen_movies = get_unseen_surprise(ratings, movies, 9)
top_movie_preds = recomment_movie_by_surprise(algorithm, 9, unseen_movies, top_n=10)

for top_movie in top_movie_preds:
    print(top_movie[1], ':', top_movie[2])
    


len(seen_movies):  46
len(seen_movies):  9696
len(seen_movies):  9742
Usual Suspects, The (1995) : 4.306302135700814
Star Wars: Episode IV - A New Hope (1977) : 4.281663842987387
Pulp Fiction (1994) : 4.278152632122758
Silence of the Lambs, The (1991) : 4.226073566460876
Godfather, The (1972) : 4.1918097904381995
Streetcar Named Desire, A (1951) : 4.154746591122658
Star Wars: Episode V - The Empire Strikes Back (1980) : 4.122016128534504
Star Wars: Episode VI - Return of the Jedi (1983) : 4.108009609093436
Goodfellas (1990) : 4.083464936588478
Glory (1989) : 4.07887165526957
